# Types and Dispatch in Julia

Julia is built around types. Software architectures in Julia are built around good use of the type system. This makes it easy to build generic code which works over a large range of types and gets good performance. The purpose of the following is to get a feeling for how types work in Julia and how they are used in what's called **multiple dispatch** to make this happen.

# Abstract vs concrete types

In [11]:
typeof(3)

Int64

In [12]:
typeof(3.0)

Float64

In [13]:
isconcretetype(Float64)

true

In [14]:
isabstracttype(Float64)

false

In [16]:
isabstracttype(Number)

true

## Duck typing

If it quacks like a duck, it might as well be a duck. This is the idea of defining an object by the way that it acts. This idea is central:

**Abstract types are defined by how they act.**

For example, a `Number` is some type that can do things like `+`,`-`,`*`, and `/`. In this category we have things like `Float64` and `Int32`. An `AbstractFloat` is some floating point number, and so it should have an implementation of `eps(T)` that gives its machine epsilon.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

**Concrete types can be instantiated and have a concrete implementation.**

Let's play around a bit and inspect the type tree.

# Inspecting the type tree

In [11]:
supertype(Float64)

AbstractFloat

In [13]:
supertype(AbstractFloat)

Real

In [16]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [14]:
supertype(Real)

Number

In [18]:
supertype(Number)

Any

Everything is a subtype of Any

In [71]:
Number <: Any

true

In [72]:
Float64 <: Any

true

In [73]:
Int32 <: Any

true

In [74]:
Int32 <: String

false

There is also `isa` for objects:

In [75]:
3.0 isa Float64

true

In [76]:
3 isa Float64

false

Let's inspect a single branch of the big Julia type tree

In [25]:
function show_supertypes(typ::DataType) 
 print(typ) 
 while typ != Any 
     typ = supertype(typ) 
     print(" <: ", typ) 
 end 
end

show_supertypes (generic function with 1 method)

In [26]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [34]:
show_supertypes(String)

String <: AbstractString <: Any

Ok, let's print a couple of branches

In [27]:
function show_subtypetree(t, level=1, indent=4)
   level == 1 && println(t)
   for s in subtypes(t)
     println(join(fill(" ", level * indent)) * string(s))
     show_subtypetree(s, level+1, indent)
   end
end

show_subtypetree (generic function with 3 methods)

In [63]:
show_subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


In [62]:
show_subtypetree(AbstractArray)

AbstractArray
    AbstractRange
        LinRange
        OrdinalRange
            AbstractUnitRange
                Base.OneTo
                Base.Slice
                UnitRange
            StepRange
        StepRangeLen
    Base.LogicalIndex
    Base.ReinterpretArray
    Base.ReshapedArray
    BitArray
    CartesianIndices
    Core.Compiler.AbstractRange
        Core.Compiler.LinRange
        Core.Compiler.OrdinalRange
            Core.Compiler.AbstractUnitRange
                Core.Compiler.OneTo
                Core.Compiler.Slice
                Core.Compiler.StmtRange
                Core.Compiler.UnitRange
            Core.Compiler.StepRange
        Core.Compiler.StepRangeLen
    Core.Compiler.BitArray
    Core.Compiler.LinearIndices
    DenseArray
        Array
        Base.CodeUnits
        Random.UnsafeView
        SharedArrays.SharedArray
        SuiteSparse.CHOLMOD.Dense
    LinearAlgebra.AbstractQ
        LinearAlgebra.QRCompactWYQ
        LinearAlgebra.QRPackedQ
        

# Dispatch

In [40]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [42]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[40]:5
[2] f(a::Number, b::Number) in Main at In[40]:2
[3] f(a::Number, b) in Main at In[40]:3
[4] f(a, b::Number) in Main at In[40]:4
[5] f(a, b) in Main at In[40]:1

In [43]:
f(1.5, 2)

"a and b are both numbers"

In [44]:
f(1, "Köln!")

"a is a number"

In [45]:
f(1, 2)

"a and b are both integers"

In [46]:
f("Hello", "World!")

"fallback"

**Julia's dispatch mechanism always chooses the most specific method for the given input types.**

One can check which particular method is being used through the `@which` macro.

In [49]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[40]:5

In [53]:
@which f(1, "Köln!")

f(a::Number, b) in Main at In[40]:3

In [47]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:349
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:347
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:385
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\s

In [56]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:104

In [60]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:229

In [61]:
methodswith(Bool)

[1] sparsevec(I::AbstractArray{T,1} where T, V::Union{Bool, AbstractArray{Bool,1}}) in SparseArrays at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\SparseArrays\src\sparsevector.jl:217
[2] sparsevec(I::AbstractArray{T,1} where T, V::Union{Bool, AbstractArray{Bool,1}}, len::Integer) in SparseArrays at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\SparseArrays\src\sparsevector.jl:220
[3] process_messages(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool) in Distributed at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Distributed\src\process_messages.jl:107
[4] process_messages(r_stream::IO, w_stream::IO, incoming::Bool) in Distributed at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Distributed\src\process_messages.jl:135
[5] send(socket::ZMQ.Socket, data, more::Bool) in ZMQ at deprecated.jl:53
[6] !(x::Bool) in Base at bool.jl:35
[7] rem(x::Bool, y::Bool) in Base at bool.jl:125
[8] rem(x::Bool, ::Type{Int8}) in Base at int.jl:460
[9] rem(x::Bool, ::Type{Int16}) in Base at int.jl:460
[10] rem(x::Bool, ::Type{Int32}) in Base at int.jl:460
[11] rem(x::Bool, ::Type{Int64}) in Base at int.jl:460
[12] rem(x::Bool, ::Type{Int128}) in Base at int.jl:460
[13] rem(x::Bool, ::Type{UInt8}) in Base at int.jl:460
[14] rem(x::Bool, ::Type{UInt16}) in Base at int.jl:460
[15] rem(x::Bool, ::Type{UInt32}) in Base at int.jl:460
[16] rem(x::Bool, ::Type{UInt64}) in Base at int.jl:460
[17] rem(x::Bool, ::Type{UInt128}) in Base at int.jl:460
[18] &(b::Bool, a::Missing) in Base at missing.jl:125
[19] &(x::Bool, y::Bool) in Base at bool.jl:40
[20] &(a::Missing, b::Bool) in Base at missing.jl:124
[21] *(x::Bool, z::Complex{Bool}) in Base at complex.jl:281
[22] *(x::Bool, y::Bool) in Base at bool.jl:106
[23] *(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:118
[24] *(x::Bool, z::Complex) in Base at complex.jl:288
[25] *(x::Bool, y::AbstractIrrational) in Base at irrationals.jl:135
[26] *(z::Complex{Bool}, x::Bool) in Base at complex.jl:282
[27] *(y::AbstractFloat, x::Bool) in Base at bool.jl:120
[28] *(z::Complex, x::Bool) in Base at complex.jl:289
[29] *(A::LinearAlgebra.Symmetric, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[30] *(A::LinearAlgebra.Hermitian, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[31] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[32] +(x::Bool, y::Bool) in Base at bool.jl:104
[33] +(x::Bool) in Base at bool.jl:101
[34] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[35] +(x::Bool, z::Complex) in Base at complex.jl:284
[36] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[37] +(y::AbstractFloat, x::Bool) in Base at bool.jl:114
[38] +(z::Complex, x::Bool) in Base at complex.jl:285
[39] -(x::Bool, z::Complex{Bool}) in Base at complex.jl:279
[40] -(x::Bool, y::Bool) in Base at bool.jl:105
[41] -(x::Bool) in Base at bool.jl:102
[42] -(x::Bool, z::Complex) in Base at complex.jl:286
[43] -(z::Complex{Bool}, x::Bool) in Base at complex.jl:280
[44] -(z::Complex, x::Bool) in Base at complex.jl:287
[45] /(A::LinearAlgebra.Symmetric, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[46] /(A::LinearAlgebra.Hermitian, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[47] <(x::Bool, y::Bool) in Base at bool.jl:96
[48] <<(x::Bool, c::Int64) in Base at bool.jl:82
[49] <<(x::Bool, c::Unsigned) in Base at bool.jl:78
[50] <<(x::Bool, c::Integer) in Base at bool.jl:8

# "Diagonal" dispatch

In [133]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [134]:
d(3, 4)

"same type"

In [135]:
d(3.0, 1.0)

"same type"

In [137]:
d(1, 4.2)

"different types"

# Parametric types

Types can have a nested structure.

In [138]:
typeof(rand(2,2))

Array{Float64,2}

In [139]:
typeof([1 2; 3 4])

Array{Int64,2}

In [140]:
typeof((1,2.0))

Tuple{Int64,Float64}

In [144]:
typeof([(1,2.0), (4,5.0)])

Array{Tuple{Int64,Float64},1}

In [149]:
Array{Float64,2} <: Array

true

In [166]:
Matrix{Float64} === Array{Float64, 2}

true

Note that parametric types have the following (somewhat counterintuitive) property

In [169]:
Array{Float64,2} <: Array{AbstractFloat,2}

false

although we have

In [170]:
Float64 <: AbstractFloat

true

The correct way to write this is

In [211]:
Array{Float64, 2} <: (Array{T, 2} where T<:AbstractFloat)

true

or shorter

In [213]:
Array{Float64, 2} <: Array{<:AbstractFloat, 2}

true

This is equivalent because we have

In [214]:
Array{<:AbstractFloat, 2} == (Array{T, 2} where T<:AbstractFloat)

true

**Quick exercise**: write a function that only takes real Matrices as input.

The following **won't work**:

In [5]:
g(x::Matrix{Real}) = "that was a matrix of real numbers"
g(x) = "wrong"

g (generic function with 3 methods)

In [6]:
g(rand(2,2))

"wrong"

In [7]:
g(rand(ComplexF64, 2,2))

"wrong"

The correct way is

In [8]:
g(x::Matrix{<:Real}) = "that was a matrix of real numbers"

# or

g(x::Matrix{T}) where T<:Real = "that was a matrix of real numbers"

g (generic function with 4 methods)

In [9]:
g(rand(2,2))

"that was a matrix of real numbers"

In [10]:
g(rand(ComplexF64, 2,2))

"wrong"

# Duck typing examples

## UnitRange

In [77]:
x = 1:30

1:30

In [80]:
typeof(x)

UnitRange{Int64}

In [81]:
typeof(x) <: AbstractArray

true

Because it is a subtype of AbstractArray I can do (some) array-like things with it, like indexing

In [82]:
x[3]

3

However, it's not a regular `Array`. In fact, it's just two numbers! We can see this by looking at it's fields:

In [93]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code (tracing the constructor)

In [94]:
@which UnitRange{Int64}(1, 10)

(::Type{UnitRange{T}})(start, stop) where T<:Real in Base at range.jl:255

It is an `immutable` type which just holds the start and stop values. This means that its indexing, `A[i]`, is just a function. What's nice about this is that means that no array is ever created. Creating large arrays can be a costly action:

In [96]:
@time collect(1:10000000);

  0.037605 seconds (7 allocations: 76.294 MiB, 26.12% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [98]:
@time 1:10000000;

  0.000001 seconds (5 allocations: 192 bytes)


Yet, in cases where we just want to index values, they act exactly the same.

## Uniform scaling operator

Another great example is the `UniformScaling` operator. It automatically gets loaded into scope when you do `using LinearAlgebra` and has the name `I`.

In [19]:
using LinearAlgebra

In [20]:
I

UniformScaling{Bool}
true*I

In [22]:
?I

search: I IO In if Int in im Inf isa Int8 inv Int64 Int32 Int16 imag Inf64 Inf32



```
I
```

An object of type [`UniformScaling`](@ref), representing an identity matrix of any size.

# Examples

```jldoctest
julia> fill(1, (5,6)) * I == fill(1, (5,6))
true

julia> [1 2im 3; 1im 2 3] * I
2×3 Array{Complex{Int64},2}:
 1+0im  0+2im  3+0im
 0+1im  2+0im  3+0im
```


Although it never actually allocates a full identity matrix it behaves like one

In [23]:
A = rand(1:10, 2,2)

2×2 Array{Int64,2}:
 10  8
  2  5

In [24]:
I * A

2×2 Array{Int64,2}:
 10  8
  2  5

In [25]:
A + I

2×2 Array{Int64,2}:
 11  8
  2  6

This can calculate expressions like `A-b*I` without ever forming the matrix `eye(n)` which would take $\mathcal{O}(n^2)$ memory. Let's benchmark the performance difference!

In [204]:
using BenchmarkTools
b = 3

@btime $A - $b * $I

  30.421 ns (1 allocation: 112 bytes)


2×2 Array{Int64,2}:
 5  5
 8  6

In [205]:
eye = Matrix(1.0I, 2,2) # alternatively but slower, diagm(0 => [1.0, 1.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [206]:
@btime $A - $b * Matrix(1.0I, 2,2)

  120.085 ns (3 allocations: 336 bytes)


2×2 Array{Float64,2}:
 5.0  5.0
 8.0  6.0

# Duck typing examples